In [1]:
import pandas as pd
import numpy as np

References:

http://doc.scrapy.org/en/latest/intro/tutorial.html

https://www.data-blogger.com/2016/08/18/scraping-a-website-with-python-scrapy/

https://cs7083.wordpress.com/2013/01/31/demystifying-the-pagerank-and-hits-algorithms/

#### Steps to write a simple webscraper in Python using the Scrapy framework

The purpose of Scrapy is to extract content and links from a website by recursively following all the links on the given website. 

Install Scrapy using command: pip install scrapy

A. Create the folder structure for your project by using Command: scrapy startproject datablogger_scraper and Update the below files as shown 

In [ ]:
# items.py
# %load homework7/datablogger_scraper/datablogger_scraper/items.py
import scrapy

class DatabloggerScraperItem(scrapy.Item):
    # The source URL
    url_from = scrapy.Field()
    # The destination URL
    url_to = scrapy.Field()

Creating first spider,Datablogger.py
You can customize this file as much as you want. I ended up with the following code,
Start_url is "data-blogger.com" and setting the DEPTH_LIMIT to 200.
In the parse function, parent url, links present in the parent web page and title are extracted.
Next if the extracted links are not already visited, then they are recursively called to scrap a website.

In [ ]:
#Creating first spider,Datablogger.py
# %load homework7/datablogger_scraper/datablogger_scraper/spiders/Datablogger.py
import scrapy
import csv
from scrapy.linkextractor import LinkExtractor
from scrapy.spiders import Rule, CrawlSpider
from datablogger_scraper.items import DatabloggerScraperItem
from scrapy.conf import settings

class DatabloggerSpider(scrapy.Spider):

    custom_settings = {'DEPTH_LIMIT': 200,}
    
    
    # The name of the spider
    name = "datablogger"
    # The domains that are allowed (links to other domains are skipped)
    allowed_domains = ["data-blogger.com"]
    
    # to keep track of already visited web pages
    linksvisited=[]
    linksvisited.append("https://www.data-blogger.com")
    linksvisited.append("https://www.data-blogger.com/")
    

    # The URLs to start with
    start_urls = ["https://www.data-blogger.com/"]
       
    
    # Method for parsing items
    def parse(self, response):
        items=[]
        links = response.css('a::attr(href)').extract()
        # append the link as visited
        DatabloggerSpider.linksvisited.append(response.url)
        linksn=[]
        if links:
           linksn= list(set(links)- set(DatabloggerSpider.linksvisited)) 
        
        for url in linksn:
           
           if url not in DatabloggerSpider.linksvisited  :
              # extract necessary information  from response
              item = []
              item.append(response.url)
              item.append(url)
              item.append(response.css('title::text').extract())
              yield scrapy.Request( url,callback=self.parse)     
             # store the information into csv
              with open("datablogger_scraper/links.csv", 'a') as myfile:
                 wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                 wr.writerow(item) 
                 

#### Executing the Spider
scrapy crawl datablogger -o links.csv -t csv

#### Data preparation for Pagerank algorithm

1. Read the collect data from 'links.csv' file into a pandas DataFrame
2. create the Stochastic matrix to perfrom the analyis

In [2]:
# read the data into DataFrame
data = pd.read_csv("datablogger_scraper/links.csv")
data.head()

,from_url,to_url,title
0,https://www.data-blogger.com/,http://twitter.com/kmjjacobs/status/8754672718...,['Data Blogger - An interesting blog with tuto...
1,https://www.data-blogger.com/,https://twitter.com/share?text=A%20Comparative...,['Data Blogger - An interesting blog with tuto...
2,https://www.data-blogger.com/,https://twitter.com/search?q=%23euler&src=hash,['Data Blogger - An interesting blog with tuto...
3,https://www.data-blogger.com/,https://www.data-blogger.com/2017/07/17/a-comp...,['Data Blogger - An interesting blog with tuto...
4,https://www.data-blogger.com/,https://www.data-blogger.com/2017/11/01/pokerb...,['Data Blogger - An interesting blog with tuto...


In [3]:
# Dertermine the unique parent links to build stochastic matrix
unique_fromurl=data['from_url'].unique()


In [4]:
# calculate the number of out going links from each parent link
links_out=data[['from_url','to_url']].groupby(['from_url'],as_index=False).count()

In [5]:
links_out.head()

,from_url,to_url
0,https://www.data-blogger.com/,154
1,https://www.data-blogger.com/2016/01/20/spam-d...,60
2,https://www.data-blogger.com/2016/01/20/the-ma...,58
3,https://www.data-blogger.com/2016/01/21/buildi...,64
4,https://www.data-blogger.com/2016/01/21/facebo...,58


In [14]:
# create connectivity matrix with nodes as links/pages and values of matrix indicating link between two website pages
Hit=np.zeros((links_out.shape[0],links_out.shape[0]))
url_from=links_out['from_url']


Below code fills the connectivity matrix for the sample.For all the links in the parent page, value for that link (row: parent column: link) in connectivity is denoted by 1 and rest denoted by 0.

In [24]:
# creating connectivity matrix 
for i in range (len(url_from)-1):
    prob=1/int(links_out.loc[[i]]['to_url']) 
    temp = data[data['from_url']==url_from[i]]['to_url']
    index = list(links_out[links_out['from_url'].isin(temp)].index)
    Hit[i][index]=1
    Hit[i][i]=0

print(Hit)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


Below is the Hits algorithm, for which will we pass will pass created connectivity matrix to get top 5 authorities and top 5 hubs urls.

Refer below link to understand hits algorithm
https://cs7083.wordpress.com/2013/01/31/demystifying-the-pagerank-and-hits-algorithms/

In [29]:
from numpy import *
def hits(A):
    n= len(A)
    Au= dot(transpose(A),A)
    Hu = dot(A,transpose(A))
    a = ones(n); h = ones(n)
    print (a,h)
    for j in range(5):
        a = dot(a,Au)
        a= a/sum(a)
        h = dot(h,Hu)
        h = h/ sum(h)
        print (a,h)
        if j==4:
            return a,h

#### call function hits on connectivity matrix and store the 5th iteration result in a and h variables, representing authorities and hubs

In [30]:
a,h=hits(Hit)

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1

#### get the top 5 authorities(in-degree) from the sample using argsort function

In [32]:
links_out.loc[np.argsort(a)[-5:]]['from_url']

18    https://www.data-blogger.com/2016/08/13/apache...
58       https://www.data-blogger.com/become-a-blogger/
60    https://www.data-blogger.com/category/data-sci...
77    https://www.data-blogger.com/category/uncatego...
32    https://www.data-blogger.com/2017/03/15/pi-day...
Name: from_url, dtype: object

####  get the top 5 Hubs(out-degree) from the sample using argsort function

In [33]:
links_out.loc[np.argsort(h)[-5:]]['from_url']

73      https://www.data-blogger.com/category/services/
67    https://www.data-blogger.com/category/mathemat...
44    https://www.data-blogger.com/2017/11/01/pokerb...
0                         https://www.data-blogger.com/
37    https://www.data-blogger.com/2017/06/15/fraud-...
Name: from_url, dtype: object